이처럼 확률을 직접 추정할때, 전체 단어를 조합하는 대신 일부 단어 조합의 출현 빈도만을 계산하여 확률을 추정하는 방법인 n-gram을 살펴보겠습니다.

### 희소성 문제

언어 모델은 문장의 확률을 수식으로 나타낼 수 있었고, 해당 확률을 근사하기 위해서는 수집한 코퍼스에서 각 단어 시퀀스의 출현 빈도를 계산하면 됩니다.

하지만 그렇다면 확률 추정치는 대부분 제대로 구하기 어렵습니다. 수많은 문장을 수집해도, 애초에 출현 가능한 단어의 조합의 경우의 수는 훨씬 더 클것이기 때문입니다. 단어들의 조합이 조금만 길어져도 코퍼스에서 출현 빈도를 구할 수 없어 분자가 0이 되면서 확률이 0이 되거나, 심지어 분모가 0이 되어 정의 자체가 불가능할 수 있습니다. 이렇게나 너무나도 많은 경우의 수때문에 생기는 문제를 희소성 문제라고 합니다.

### 마르코프 가정

코퍼스에서 단어 시퀀스에 대한 확률을 효과적으로 추정하려면 희소성 문제를 해결해야 합니다. 이때 Markov Assumption을 도입합니다.

Markov Assumption이란 특정 시점의 상태 확률은 단지 그 직전 상태에만 의존한다는 논리입니다. 즉, 앞서 출현한 모든 단어를 살펴볼 필요 없이, 앞의 k개의 단어만 보고 다음 단어의 출현 확률을 구하는 것입니다. 즉, 앞서 출현한 모든 단어를 살펴볼 필요 없이, 앞의 k개의 단어만 보고 다음 단어의 출현 확률을 구하는 것입니다.

<br></br>
$$
P(x_i|x_1, x_2, \dots, x_{i-1}) \approx P(x_i|x_{i_-k}, \dots, x_{i-1})
$$
<br></br>

이렇게 조건을 간소화하여 실제로 구하고자 하는 확률을 근사합니다. 보통 k는 0에서 3의 값을 갖습니다. 즉, k=2일 경우에는 앞 단어 2개를 참조하여 다음 단어 $(x_i)$의 확률을 근사하여 나타냅니다.

<br></br>
$$
P(x_i|x_{i-2}, x_{i-1})
$$
<br></br>

여기에 연쇄벅칙을 적용한 문장에 대한 확률도 다음과 같이 표현할 수 있습니다.

<br></br>
$$
P(x_1, x_2, \dots, x_n) \approx \prod_{i=1}^n P(x_i|x_{i-k}, \dots, x_{i-1})
$$
<br></br>

이것읋 로그 확률로 표현하면 다음과 같습니다.

<br></br>
$$
log P(x_1, x_2, \dots, x_n) \approx \sum_{i=1}^n log P(x_i|x_{i-k}, \dots, x_{i-1})
$$
<br></br>

이렇게 전체 단어를 조합하는 대신 바로 앞의 일부 조합만 출현 빈도를 계산하여 확률을 추정하는 방법을 n-gram이라고 부릅니다. 이때 n=k+1로 나타냅니다.

<br></br>

|k|n-gram|명칭|
|-|------|---|
|0|1-gram|uni-gram|
|1|2-gram|bi-gram|
|2|3-gram|tri-gram|

<br></br>

훈련 코퍼스의 양이 작을수록 n의 크기도 작아져야 합니다. 길고 복잡한 단어 시퀀스일수록 훈련 코퍼스에 등장하지 않을 가능성이 높기 때문입니다. 보통은 대부분 훈련 코퍼스의 양이 적다는 가정하에 3-gram을 가장 많이 사용하며, 훈련 코퍼스의 양이 많을때는 4-gram을 사용하기도 합니다.

3-gram의 가정에 따라, 다음과 같이 3개 단어의 출현 빈도와 앞 2개 단어의 출현 빈도만 구하면 $x_i$의 확률을 근사할 수 있습니다.

<br></br>
$$
P(x_i|x_{i-2}, x_{i-1}) = \frac{Count(x_{i-2},x_{i-1},x_i)}{Count(x_{i-2},x_{i-1})}
$$
<br></br>

즉, 문장 전체의 확률에 대해 Markov Assumption을 도입하여 해당 문장의 확률을 근사할 수 있게 되었습니다. 이제는 훈련 코퍼스에서 보지 못한 문장에 대해서도 확률을 추정할 수 있습니다.

### 일반화

좋은 머신러닝은 훈련 데이터에서 보지 못한 샘플의 예측 능력, 즉 일반화 능력에 좌우합니다. n-gram 역시 Markov Assumption의 도입을 통해 희소성에 대처하는 일반화 능력을 어느정도 갖추었다고 볼 수 있습니다.

#### 스무딩과 디스카운팅

출현 횟수를 단순히 확률값으로 추정할 경우의 문제점은 무엇일까요?

바로 훈련 코퍼스에 출현하지 않은 단어 시퀀스에 대한 대처 능력입니다. 앞서 설명했듯이 마르코프 가정을 적용하여 희소성 문제를 훨씬 줄일수 있었고, 문장의 모든 단어 조합에 대해서 출현 횟수를 세지 않아도 되지만, 훈련 코퍼스에 등장하지 않는 경우에는 여전히 존재할것입니다. 훈련 코퍼스에서 **$보지\ 못한\ 단어\ 조합^{unseen\ word\ sequence}$**이라고 해서 확률을 0으로 추정할 수는 없습니다. 따라서 출현 빈도값 또는 확률값을 더 다듬어줘야 합니다.

쉽게 말하자면, 그 과정에서 들쭉날쭉하던 출현 횟수의 값이 부드럽게 바뀌는 것을 가리켜 $스무딩^{smoothing}$ 또는 $디스카운팅^{discounting}$이라고 합니다.

<br></br>
![](./images/9-2-3-smoothing.jpg)
<br></br>

먼저 생각해볼 수 있는 가장 간단한 방법은, 모든 단어 시퀀스의 출현 빈도에 1을 더하는 것입니다. 그럼 확률값이 0이 되는 것을 막아주며, 훈련 코퍼스에 출현하지 않은 n-gram의 경우에도 작은 확률이나마 가질 수 있습니다.

<br></br>
$$
P(w_i|w_{<i}) \approx \frac{Count(w_{<i},w_i)+1}{Count(w_{<i})+V}
$$
<br></br>

이처럼 1을 더하면 $P(w_i|w_{<i})$를 더 효과적으로 근사할 수 있습니다. 더 일반화하여 표현하자면:

<br></br>
$$
P(w_i|w_{<i}) \approx \frac{Count(w_{<i},w_i)+k}{Count(w_{<i})+kV} \\
\approx \frac{Count(w_{<i},w_i)+(\frac{m}{V})}{Count(w_{<i})+m}
$$
<br></br>

이와 같이 1보다 작은 상수값을 더하여 스무딩을 구현해볼 수 있습니다. 나아가 1-gram의 사전 확률을 이용하여 더욱 저극적으로 대처해볼수 있습니다.

<br></br>
$$
P(w_i|w_{<i}) \approx \frac{Count(w_{<i},w_i)+mP(w_i)}{Count(w_{<i})+m} \\
$$
<br></br>

이 방법은 매우 간단하고 직관적이지만, 언어 모델처럼 희소성 문제가 클 경우 사용하기에 적합하지 않습니다.

#### Kneser-Ney 디스카운팅

언어 모델의 성능을 높이기 위한 방법으로 **KN 디스카운팅**이 존재합니다. KN 디스카운팅의 핵심 아이디어는 단어 w가 다른 단어 v의 뒤에 출현할대 얼마나 다양한 단어 뒤에서 출현하는지 (즉, v가 얼마나 다양한지)를 알아내는 것입니다. 다양한 단어 뒤에 나타나는 단어일수록 훈련 코퍼스에서 보지 못한 단어 시퀀스로 나타날 가능성이 높다는 내용입니다.

KN 디스카운팅은 $Score_{continuation}$을 다음과 같이 모델링합니다. 즉, w와 함께 나타난 v들의 집합 {w:Count(v,w)>}의 크기가 클수록 $Score_{continuation}$은 클 것이라고 가정합니다.

<br></br>
$$ Score_{continuation}(w) \propto |\{v: Count(v,w) > 0\}| $$
<br></br>

w와 함께 나타난 v들의 집합 {v:Count(v,w) > 0}의 크기를 나타내고, 전체 단어 집합으로부터 샘플링한 $w' \in W$일때, v, w'가 함께 나타난 집합 {v:Count(v,w' > 0}의 크기의 합으로 나눕니다.

<br></br>
$$ Score_{continuation}(w) = \frac{|\{v: Count(v,w) > 0\}|}{\sum_{w'}|\{v: Count(v,w') > 0\}|} $$
<br></br>

이렇게 우리는 bi-gram을 위한 $P_{KN}$을 다음 수식에서 정의할 수 있습니다. 이때 d는 상수로써 보통 0.75의 값을 가집니다.

<br></br>
$$
P_{KN}(w_i|w_{i-1}) = \frac{max(Count(w_{i-1},w_i)-d,0)}{Count(w_{i-1})} + \lambda(w_{i-1})  Score_{continuation}(w_i) \\
\text{where}\ \lambda(w_{i-1}) = \frac{d}{\sum_v Count(w_{i-1})} |\{ w:c(w_{i-1},v)>0 \} |
$$
<br></br>

이처럼 KN 디스카운팅은 간단한 직관에서 출발하여 복잡한 수식을 갖습니다. 여기서 약간의 수정이 더해진 `Modified-KN Discounting`이 가장 널리 쓰이는 방법입니다.

#### 인터폴레이션

다수의 언어 모델 사이의 선형결합, 즉 인터폴레이션을 통하여 언어 모델을 일반화하는 방법을 살펴보겠습니다. 언어 모델의 인터폴레이션은 두개의 다른 언어 모델을 선형적으로 일정 비율 ($\lambda$)로 섞어주는 것입니다. 특정 영역에 특화된 언어 모델을 구축하려 할때 인터폴레이션은 매우 유용합니다. 일반 영억의 코퍼스를 통해 구축한 언어 모델을 특정 영역의 작은 코퍼스로 만든 언어 모델과 섞어줌으로써 특정 영억에 특화된 언어 모델을 강화할 수 있습니다.

<br></br>
$$
\tilde{P}(w_n|w_{n-k}, \dots, w_{n-1}) = \lambda P_1(w_n|w_{n-k}, \dots, w_{n-1}) + (1-\lambda) P_2(w_n|w_{n-k}, \dots, w_{n-1})
$$
<br></br>

예를 들어 의료 분야의 음성 인식 또는 기계번역 시스템을 구축한다고 가정해본다면, 기존의 일반 영역 코퍼스를 통해 생성한 언어 모델이라면 의료 용어 표현이 낯설 수 있습니다. 반대로 특화 영역의 코퍼스만 사용하여 언어 모델을 생성한다면 일반화 능력이 지나치게 떨어질 수 있습니다.

<br></br>
* **일반 영역**
    * P(진정제|준비,된) = 0.00001
    * P(사나이|준비,된) = 0.01
    
* **특화 영역**
    * P(진정제|준비,된) = 0.09
    * P(약|준비,된) = 0.04
    
* **인터폴레이션 결과**
    * P(진정제|준비,된) = 0.5 x 0.09 + (1-0.5)x0.00001 = 0.045005
<br></br>

결국 일반적인 의미와는 다른 뜻의 단어가 나올 수도 있고, 일반적인 대화에서는 희소한 단어 시퀀스가 훨씬 자주 등장할 수도 있습니다. 또한 특화 영역의 코퍼스에는 일반적인 단어 시퀀스가 매우 부족할 것입니다. 이런 문제점들을 해결하기 위해 각 영역의 코퍼스로 생성한 언어 모델을 섞어주어 해당 영역에 특화할 수 있습니다.

#### 백오프

너무 길거나 복잡한 단어 시퀀스는 실제 훈련 코퍼스에 굉장히 희소하므로 마르코프 가정을 통해서 일반화할 수 있었습니다.

Back-off 방식은 여기에서 한발 더 나아갑니다. 다음 수식을 보면 특정 n-gram의 확률을 n보다 더 작은 시퀀스에 대해 확률을 구하여 인터폴레이션합니다. 예를 들어 3-gram의 확률에 대해 2-gram이나 1-gram의 확률을 인터폴레이션할때, 이를 수식으로 나타내면 다음과 같이 n보다 더 작은 시퀀스의 확률을 활용함으로써 더 높은 스무딩 및 일반화 효과를 얻을 수 있습니다.

<br></br>
$$
\tilde{P}(w_n|w_{n-k}, \dots, w_{n-1}) = \lambda_1 P(w_n|w_{n-k}, \dots, w_{n-1}) \\
+ \lambda_2 P(w_n|w_{n-k+1}, \dots, w_{n-1}) \\
+ \dots \\
+ \lambda_k P(w_n) \\
\text{where}\ \sum_i \lambda_i = 1
$$
<br></br>